__BIOBSS - Polar reader__

_This notebook includes guidelines to save Polar data files as csv file and and synchronize the signal from different sensors._

In [1]:
#Import BIOBSS and the other required packages

import biobss
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Table of Contents
1.[Polar Sample Data](#polar)<br>
2.[Timestamp Conversion](#polar_time)<br>
3.[Synchronization of Signals from Different Sensors](#polar_sync)<br>
4.[Data Segmentation Based on Events](#polar_event)<br>
5.[Loading and Saving Alternatives](#polar_files)<br>

### __Polar Sample Data__
<a id="polar"></a>

Polar sample data is provided as txt files in BIOBSS\sample data\Polar_files\txt_files. A seperate txt file is available for each sensor. The sampling rates are 52 Hz, 52 Hz, 50 Hz and 55 Hz for ACC, GYRO, MAGN and PPG sensors respectively. 

In [2]:
#Define the paths for the sample files
polar_dir=os.path.join(os.path.normpath(os.getcwd() + os.sep + os.pardir),"sample_data","Polar_files")
txt_dir=os.path.join(polar_dir,'txt_files')
csv_dir=os.path.join(polar_dir,'csv_files')
pkl_dir=os.path.join(polar_dir,'pkl_files')

Txt files can be saved as csv files into BIOBSS\sample_data\Polar_files\csv_files. If required, the files can be renamed using the function ___rename_csv___.

In [3]:
#Save txt files as csv files
biobss.reader.polar_format.txt_to_csv(txt_dir)

#Rename csv files using the record_id
biobss.reader.polar_format.rename_csv(csv_dir)

c:\Users\ipek.karakus\.conda\envs\biolib\lib\site-packages\biobss\reader\polar_format.py:72: UserWarning: No files found in c:\Users\ipek.karakus\Desktop\biobss\sample_data\Polar_files\txt_files.
  warnings.warn(f'No files found in {root}.')
c:\Users\ipek.karakus\.conda\envs\biolib\lib\site-packages\biobss\reader\polar_format.py:70: UserWarning: c:\Users\ipek.karakus\Desktop\biobss\sample_data\Polar_files\csv_files\sample_record already present, skipping txt to csv conversion.
  warnings.warn(f'{csv_dir} already present, skipping txt to csv conversion.')
c:\Users\ipek.karakus\.conda\envs\biolib\lib\site-packages\biobss\reader\polar_format.py:110: UserWarning: No files found in c:\Users\ipek.karakus\Desktop\biobss\sample_data\Polar_files\csv_files.
  warnings.warn(f'No files found in {csv_dir}.')


### __Timestamp Conversion__
<a id="polar_time"></a>

Timestamps in iso format can be converted to milliseconds for a specific file. If required, the reference timestamp can be provided by the parameter __start_time__. Otherwise, the first timestamp is used as reference.

In [5]:
#Convert timestamps from iso format to milliseconds for a specific file
record_id='sample_record'
filename='sample_record_PPG.csv'
filepath=os.path.join(csv_dir,record_id,filename)
df= pd.read_csv(filepath)
time_msec=biobss.reader.polar_format.timestamp_to_msec(df['Phone timestamp'])

The ___add_record_time___ function can be used to update the selected csv files by adding a 'Time_record (ms)' column corresponding to time points in ms. The time points are calculated referenced to the earliest timestamp for all sensors.

In [6]:
#Convert timestamps from iso format to milliseconds for all files in a record
record_id='sample_record'
csv_subdir=os.path.join(csv_dir,record_id) #This time, the path should be defined for a specific record.
biobss.reader.add_record_time(csv_dir=csv_subdir,marker=True)

### __Synchronization of Signals from Different Sensors__
<a id="polar_sync"></a>

Signals from different sensors can be synchronized using the time points. First, a time array is generated by detecting the overlapping time points for all sensors. Then, the 'Time_record (ms)' column of each sensor is interpolated on the generated time array. If required, signals can also be resampled at this stage. 

In [7]:
#Generate a time array considering the overlapping time points for all sensors. 
#The path should be defined for a specific record.
#If required, save the generated time array as a txt file by setting save_file=True
time_list=biobss.reader.calculate_sync_time(csv_dir=csv_subdir,time_step=1,marker=True)

In [8]:
#Synchronize the signals from different sensors by interpolating the signals for the given time_list.
#Resample the signals if required by providing resampling_rate 
#Set save_files=True to save the synchronized signals as a csv file
data=biobss.reader.synchronize_signals(csv_subdir,time_list=time_list,resampling_rate=100,save_files=True)

### __Data Segmentation Based on Events__
<a id="polar_event"></a>

If required, sensor data can be segmented for different events. For this purpose, a marker file is needed. Using Polar Sensor Logger App, markers can be placed during recording and a marker file can be exported. BIOBSS provides ___marker_to_events___ function to generate an event list from the marker timestamps. 

In [10]:
#Segment the synchronized signals for events
filename = 'sync_ACC_PPG_MAGN_GYRO.csv'
filepath=os.path.join(csv_dir, record_id, filename)
markerpath=os.path.join(csv_dir, record_id, 'MARKER_'+record_id+'.csv')

data=biobss.reader.marker_to_events(filepath,markerpath,['rest'])

In [11]:
#The event list can also be saved as txt file for later use
out_path=os.path.join(csv_dir, record_id, 'event_list.txt' )
biobss.reader.marker_to_events(filepath,markerpath,['rest'],out_path,save_file=True)

,Time_record (ms),X [mg],Y [mg],Z [mg],channel 0,channel 1,channel 2,ambient,X [G],Y [G],Z [G],X [dps],Y [dps],Z [dps],Events
0,60060.000000,-526.585136,55.727686,-853.177568,250388.301633,263532.324833,280800.385432,381.703351,-1.152701,-0.437369,0.381853,1.574974,-3.660218,-2.235546,rest
1,60069.999896,-625.533960,22.783335,-799.463856,252040.606892,265592.874472,282418.720436,225.985948,-1.196616,-0.274666,0.330571,3.730015,-5.475692,-5.285922,rest
2,60079.999792,-599.400215,27.346212,-814.782651,251684.148115,265136.987000,282042.764604,298.570529,-1.181697,-0.321914,0.350219,3.845627,-3.809937,-3.836145,rest
3,60089.999688,-612.463451,21.234104,-807.723293,251928.243205,265458.040082,282275.541016,230.199431,-1.185326,-0.300478,0.345979,4.795835,-2.841006,-3.874287,rest
4,60099.999584,-600.476891,24.310629,-813.420737,251745.730992,265249.777178,282092.771449,193.561620,-1.182115,-0.324994,0.356622,5.318753,-1.352278,-3.033832,rest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48097,541025.000520,-428.195491,88.392594,-910.889153,248635.809376,261422.041061,279164.289565,626.068987,-1.098664,-0.619816,0.441665,-0.066984,-1.216455,1.289091,rest
48098,541035.000416,-435.345796,88.028704,-905.910860,248762.440784,261574.621259,279276.018839,601.948650,-1.105475,-0.605655,0.436877,0.130918,-1.566369,0.966973,rest
48099,541045.000312,-423.847690,92.451562,-911.053143,248617.159787,261393.275394,279105.052543,614.165347,-1.102484,-0.622649,0.444154,-0.094809,-1.212794,1.399047,rest
48100,541055.000208,-437.546494,86.508435,-901.198165,248872.456402,261697.601615,279330.639753,574.232155,-1.110673,-0.595695,0.437026,0.079811,-1.494375,0.942646,rest


### __Loading and Saving Alternatives__
<a id="polar_files"></a>

BIOBSS provides loading and saving alternatives for the Polar sensor files. Csv files can be read into dictionaries using the ___polar_csv_reader___ function by setting the signal_type parameter. 

Csv files can also be saved as pkl files using the ___csv_to_pkl___ function.

In [15]:
#Read csv files into a dictionary
filename='sample_record_PPG.csv'
filepath=os.path.join(csv_dir,record_id,filename)
data=biobss.reader.polar_csv_reader(filepath,signal_type='PPG')
print(data.keys())

#Save signals as pkl files
biobss.reader.csv_to_pkl(csv_dir)

dict_keys(['Timestamp', 'Time_sensor (ms)', 'Time_record (ms)', 'PPG_Ch0', 'PPG_Ch1', 'PPG_Ch2', 'Ambient'])


c:\Users\ipek.karakus\.conda\envs\biolib\lib\site-packages\biobss\reader\polar_format.py:481: UserWarning: No files found in c:\Users\ipek.karakus\Desktop\biobss\sample_data\Polar_files\csv_files.
  warnings.warn(f'No files found in {csv_dir}.')
c:\Users\ipek.karakus\.conda\envs\biolib\lib\site-packages\biobss\reader\polar_format.py:478: UserWarning: c:\Users\ipek.karakus\Desktop\biobss\sample_data\Polar_files\pkl_files\sample_record already present, skipping csv to pkl conversion.
  warnings.warn(f'{pkl_dir} already present, skipping csv to pkl conversion.')
